In [2]:
import numpy as np
import pandas as pd
import re

In [29]:
#opening file with inhibitor information
df = pd.read_csv('kinase_inhibitor_list.csv',index_col=0)
                 
#Assigning an index column to dataframe
df = df.set_index("Inhibitor", drop = False)

#removing data with no reference and no Action
df = df.dropna(subset=['Reference'])
df = df.dropna(subset=['Action'])

df.head()


,Inhibitor,Brutto,MW,Action,Reference,Reference_2,PMID2,Patent,Commercial Vendor,CAS,InChI,InChI Key,PubChem CID,SMILES,Alt Name
Inhibitor,,,,,,,,,,,,,,,
(5Z)-7-Oxozeaenol,(5Z)-7-Oxozeaenol,C19H22O7,362.37,TAK1,Ninomiya-Tsuji (2003) J.Biol.Chem. 278 18485,Windheim et al (2007) Biochem.J. 404 179,17348859.0,NaN,Tocris,66018-38-0,InChI=1S/C19H22O7/c1-11-5-3-7-14(20)18(23)15(2...,NEQZWEXWOFPKOT-UHFFFAOYSA-N,53442201.0,CC1CC=CC(=O)C(C(CC=CC2=CC(=CC(=C2C(=O)O1)O)OC)O)O,LL-Z-1640-2
1 NA-PP1 (PP1 Analog),1 NA-PP1 (PP1 Analog),C19H19N5,317.40,Src,Bishop A. C. (2000) Nature 407 395,Bishop A.C. (1999) J. Am. Chem. Soc. 121 627,NaN,WO2003082341,Merck (Calbiochem),221243-82-9,"InChI=1S/C19H19N5/c1-19(2,3)24-18-15(17(20)21-...",XSHQBIXMLULFEV-UHFFFAOYSA-N,4877.0,CC(C)(C)N1C2=C(C(=N1)C3=CC=CC4=CC=CC=C43)C(=NC...,"1-NA-PP1, CHEBI:52310, 1-Na PP1"
1 NM-PP1 (PP1 Analog II),1 NM-PP1 (PP1 Analog II),C20H21N5,331.40,Src,Ira G. (2004) Nature 431 1011,Papa F.R. (2003) Science 302 1533,14564015.0,WO2000042042,Merck (Calbiochem),221244-14-0,"InChI=1S/C20H21N5/c1-20(2,3)25-19-17(18(21)22-...",GDQXJQSQYMMKRA-UHFFFAOYSA-N,5154691.0,CC(C)(C)N1C2=C(C(=N1)CC3=CC=CC4=CC=CC=C43)C(=N...,"PP1 Analog II, 1NM-PP1, Mutant Kinases Inhibit..."
A-443654,A-443654,C24H23ON5x2HCl,470.39,Akt1,Shi Y. (2005) Neoplasia 7 992-1000.,NaN,NaN,WO2007106884,AdooQ Bioscience,552325-16-3,InChI=1S/C24H23N5O/c1-15-22-10-16(6-7-24(22)29...,YWTBGJGMTBHQTM-IBGZPJMESA-N,10172943.0,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CNC5=...,"SHN-414, A443654, DNC006711, CHEMBL379300, MLS..."
A-769662,A-769662,C20H12N2O3S,360.39,AMPK Activator,Cool B. (2006) Cell Metabolism 3 403-416,NaN,NaN,US7119205,Tocris,844499-71-4,InChI=1S/C20H12N2O3S/c21-9-14-18(24)17-15(10-2...,CTESJDQKVOEUOY-UHFFFAOYSA-N,54708532.0,C1=CC=C(C(=C1)C2=CC=C(C=C2)C3=CSC4=C3C(=C(C(=O...,"SHN-540, A769662, SureCN2669403, CHEMBL512504,..."


In [30]:
#dataframe of only inhibitor and action
My_df = pd.DataFrame(data={
  'Inhibitor':list(df.Inhibitor),
  'Action': list(df.Action)})

#one inhibitor can have many actions
#making repeats
#allocating each action to its inhibitor
 

Action_Inhibitor= (My_df.set_index(['Inhibitor'])
   .stack()
   .str.split(',', expand=True)
   .stack()
   .unstack(-2)
   .reset_index(-1, drop=True)
   .reset_index())

#248 rows
Action_Inhibitor.head()

,Inhibitor,Action
0,(5Z)-7-Oxozeaenol,TAK1
1,1 NA-PP1 (PP1 Analog),Src
2,1 NM-PP1 (PP1 Analog II),Src
3,A-443654,Akt1
4,A-769662,AMPK Activator


In [31]:
#upper case the action column 

def mod_upper(q):
    if isinstance(q,str):
        return q.upper()
    else:
        return q
    
upper_Action = [mod_upper(f) for f in Action_Inhibitor.Action]
upper_Action[:5]

['TAK1', 'SRC', 'SRC', 'AKT1', 'AMPK ACTIVATOR']

In [32]:
#some Action contains spaces before the word
#need to remove this
new_Action= []

for i in upper_Action:
    j = i.replace(' ','')
    new_Action.append(j)
    
new_Action[:10]


['TAK1',
 'SRC',
 'SRC',
 'AKT1',
 'AMPKACTIVATOR',
 'ALK5',
 'PDGF',
 'EGFR',
 'EGFR',
 'JAK']

In [33]:
Action_Inhibitor.head()

,Inhibitor,Action
0,(5Z)-7-Oxozeaenol,TAK1
1,1 NA-PP1 (PP1 Analog),Src
2,1 NM-PP1 (PP1 Analog II),Src
3,A-443654,Akt1
4,A-769662,AMPK Activator


In [34]:
#replace old action column with modidied action column 

#name of the column by index
n = Action_Inhibitor.columns[1]

# Drop old action column
Action_Inhibitor.drop(n, axis = 1, inplace = True)

# Put new column
Action_Inhibitor[n] = new_Action

Action_Inhibitor.head()

,Inhibitor,Action
0,(5Z)-7-Oxozeaenol,TAK1
1,1 NA-PP1 (PP1 Analog),SRC
2,1 NM-PP1 (PP1 Analog II),SRC
3,A-443654,AKT1
4,A-769662,AMPKACTIVATOR


In [35]:
#opening human kinase file 
human_kinase_db = pd.read_csv("gene_names.csv", index_col = 0)

#dropping Protein_name column
human_kinase_db.drop(human_kinase_db.columns[1], axis = 1, inplace = True)
human_kinase_db.head()

,Gene_name,Gene_aliases
Uniprot_number,,
P31749,AKT1,"['AKT1', 'PKB', 'RAC']"
P31751,AKT2,['AKT2']
Q9Y243,AKT3,"['AKT3', 'PKBG']"
Q5VT25,CDC42BPA,"['CDC42BPA', 'KIAA0451']"
Q9Y5S2,CDC42BPB,"['CDC42BPB', 'KIAA1124']"


In [36]:
#removing the [] from gene-aliases column
g = [i.replace('[', '') for i in human_kinase_db.Gene_aliases]
y = [i.replace(']', '') for i in g]

#removing '' quotation marks from each gene
import ast
q = str(y).replace("'", "")
q = ast.literal_eval(q)


#replacing the gene-aliales column with new column without []
#name of the column by index
n = human_kinase_db.columns[1]

# Drop that column
human_kinase_db.drop(n, axis = 1, inplace = True)

# Put new column
human_kinase_db[n] = q


#separating each gene_aliases into its respective gene_name
wanted_kinase = human_kinase_db.apply(lambda x :x.str.split(',',expand=True).stack()).reset_index(level=1,drop=True).ffill()

#1326 rows
wanted_kinase.head()

,Gene_name,Gene_aliases
Uniprot_number,,
A0A0B4J2F2,SIK1B,SIK1B
O00141,SGK1,SGK1
O00141,SGK1,SGK
O00238,BMPR1B,BMPR1B
O00311,CDC7,CDC7


In [37]:
#some gene aliases contain space before the word
#need to remove the spaces first
new_gene_aliases = []

for i in wanted_kinase.Gene_aliases:
    j = i.replace(' ','')
    new_gene_aliases.append(j)

    
##replacing gene aliases column

#name of the column by index
n = wanted_kinase.columns[1]

# Drop that column
wanted_kinase.drop(n, axis = 1, inplace = True)

# Put new column
wanted_kinase[n] = new_gene_aliases

#the new column
wanted_kinase.Gene_aliases[:10]


Uniprot_number
A0A0B4J2F2     SIK1B
O00141          SGK1
O00141           SGK
O00238        BMPR1B
O00311          CDC7
O00311        CDC7L1
O00329        PIK3CD
O00418         EEF2K
O00444          PLK4
O00444           SAK
Name: Gene_aliases, dtype: object

In [38]:

for x in wanted_kinase.Gene_aliases:  #gene aliases
    for y in Action_Inhibitor.Action: #action
        
    #if the value of gene aliase and action are the same:
        if x == y:
            
            #find the correct gene name associated with that gene aliase
            gene = wanted_kinase.loc[wanted_kinase.Gene_aliases == x, "Gene_name"].iloc[0]
            
            #replace the action column with the correct name
            Action_Inhibitor['Action'] = Action_Inhibitor['Action'].str.replace(y, gene)     
        else:
            pass


#248 rows
Action_Inhibitor.head()

            

,Inhibitor,Action
0,(5Z)-7-Oxozeaenol,MAP3K7
1,1 NA-PP1 (PP1 Analog),SRC
2,1 NM-PP1 (PP1 Analog II),SRC
3,A-443654,AKT1
4,A-769662,PRKAA2ACTIVATOR


In [39]:
#comparing Action column with gene_name column and finding its inhibitor
#if gene_name and gene_aliases match, extract its inhibitor

test=wanted_kinase.join(Action_Inhibitor.set_index('Action'), on='Gene_name')
new_db = test[test['Inhibitor'].notnull()]                    

new_db.head()


,Gene_name,Gene_aliases,Inhibitor
Uniprot_number,,,
O00141,SGK1,SGK1,GSK650394A
O00141,SGK1,SGK1,SGK-Sanofi-14i
O00141,SGK1,SGK1,SGK1-Sanofi-14g
O00141,SGK1,SGK1,SGK1-Sanofi-14h
O00141,SGK1,SGK1,SGK1-Sanofi-14n


In [40]:
#removing Gene_aliases column for next step
new_db.drop(new_db.columns[1], axis = 1, inplace = True)

new_db.head()  ##251 rows

,Gene_name,Inhibitor
Uniprot_number,,
O00141,SGK1,GSK650394A
O00141,SGK1,SGK-Sanofi-14i
O00141,SGK1,SGK1-Sanofi-14g
O00141,SGK1,SGK1-Sanofi-14h
O00141,SGK1,SGK1-Sanofi-14n


In [42]:
#remove duplicate rows
new_db = new_db.drop_duplicates()  

#105 rows
new_db.head()


,Gene_name,Inhibitor
Uniprot_number,,
O00141,SGK1,GSK650394A
O00141,SGK1,SGK-Sanofi-14i
O00141,SGK1,SGK1-Sanofi-14g
O00141,SGK1,SGK1-Sanofi-14h
O00141,SGK1,SGK1-Sanofi-14n


In [43]:
#incase of duplicates in inhibitor: group by Inhibitors
new_db.groupby('Inhibitor')['Gene_name'].apply(lambda x: x.unique()).reset_index()

#105 rows
new_db.head()

,Gene_name,Inhibitor
Uniprot_number,,
O00141,SGK1,GSK650394A
O00141,SGK1,SGK-Sanofi-14i
O00141,SGK1,SGK1-Sanofi-14g
O00141,SGK1,SGK1-Sanofi-14h
O00141,SGK1,SGK1-Sanofi-14n


In [47]:
#join two files together.
#match inhibitors from new_db and original df to extract other information

test=new_db.join(df.set_index('Inhibitor'), on='Inhibitor')
match_info = test[test['Action'].notnull()]   #testing on Action since it is guranteed to be there for every inhibitor in this df. 

#105 rows
match_info.head()

,Gene_name,Inhibitor,Brutto,MW,Action,Reference,Reference_2,PMID2,Patent,Commercial Vendor,CAS,InChI,InChI Key,PubChem CID,SMILES,Alt Name
Uniprot_number,,,,,,,,,,,,,,,,
O00141,SGK1,GSK650394A,C25H22N2O2,382.45,SGK,Sherk A. B. (2008) Cancer Res 68 7475–7483,NaN,NaN,NaN,Tocris,890842-28-1,InChI=1S/C25H22N2O2/c28-25(29)20-11-10-18(12-2...,WVSBGSNVCDAMCF-UHFFFAOYSA-N,25022668.0,C1CCC(C1)C2=C(C=CC(=C2)C3=CNC4=NC=C(C=C34)C5=C...,"SHN-954, GSK 650394A, GSK-650394A"
O00141,SGK1,SGK-Sanofi-14i,C18H15ClN6O3S,430.87,SGK,"Halland, N (2015) ACS Medicinal Chemistry Lett...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SHN-2122
O00141,SGK1,SGK1-Sanofi-14g,C17H12Cl2N6O2S,435.29,SGK,"Halland, N (2015) ACS Medicinal Chemistry Lett...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SHN-2121
O00141,SGK1,SGK1-Sanofi-14h,C17H12Cl2N6O2S,435.29,SGK,"Halland, N (2015) ACS Medicinal Chemistry Let...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SHN-2217
O00141,SGK1,SGK1-Sanofi-14n,C17H12CIFN6O2S,418.83,SGK,"Halland, N (2015) ACS Medicinal Chemistry Lett...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SHN-2108


In [48]:
#making final dataframe

My_df = pd.DataFrame({
  'Inhibitor': list(match_info.Inhibitor),'Target': list(match_info.Gene_name),
    'MW':list(match_info.MW), "Emperical Formula": list(match_info.Brutto)})


#rearranging the columns
df_final = My_df[['Inhibitor', 'Target', 'MW', 'Emperical Formula']]
df_final.head()


,Inhibitor,Target,MW,Emperical Formula
0,GSK650394A,SGK1,382.45,C25H22N2O2
1,SGK-Sanofi-14i,SGK1,430.87,C18H15ClN6O3S
2,SGK1-Sanofi-14g,SGK1,435.29,C17H12Cl2N6O2S
3,SGK1-Sanofi-14h,SGK1,435.29,C17H12Cl2N6O2S
4,SGK1-Sanofi-14n,SGK1,418.83,C17H12CIFN6O2S


In [46]:
#df into a csv file

df_final.to_csv('Incomplete_inhibitor.csv', header=True, index=False)